### Import

In [3]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve

import lightgbm as lgb
import xgboost as xgb

from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline

### Data Load

In [ ]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

train = pd.read_csv('./data/train.csv').drop(columns=['ID'])
test = pd.read_csv('./data/test.csv').drop(columns=['ID'])

X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

# 수치형 컬럼들을 0으로 채움
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

# 스케일링 적용
scaler = StandardScaler()
X_train_encoded[numeric_columns] = scaler.fit_transform(X_train_encoded[numeric_columns])
X_test_encoded[numeric_columns] = scaler.transform(X_test_encoded[numeric_columns])

# 데이터 불균형 확인 및 SMOTE 적용
smote = SMOTE(random_state=42)
X_train_encoded, y = smote.fit_resample(X_train_encoded, y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_train_encoded, y, test_size=0.1, random_state=42)

# 결과 저장 딕셔너리
auc_scores = {}
training_times = {}

# # 0. Extra Trees Classifier
# start_time = time.time()
# ETC_model = ExtraTreesClassifier(random_state=42)
# ETC_model.fit(X_train, y_train)
# training_times['Extra Trees'] = time.time() - start_time
# ETC_pred_proba = ETC_model.predict_proba(X_test)[:, 1]
# auc_scores['Extra Trees'] = roc_auc_score(y_test, ETC_pred_proba)

# # 1. Random Forest
# start_time = time.time()

# # RandomizedSearchCV를 통한 랜덤 포레스트 튜닝
# rf_params = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [10, 20, 30, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'bootstrap': [True, False]
# }

# rf_random_search = RandomizedSearchCV(
#     estimator=RandomForestClassifier(random_state=42),
#     param_distributions=rf_params,
#     n_iter=10,
#     cv=3,
#     scoring='roc_auc',
#     random_state=42
# )

# # Random Forest 튜닝 및 학습
# rf_random_search.fit(X_train, y_train)
# rf_model = rf_random_search.best_estimator_

# rf_pred_proba = rf_model.predict_proba(X_test_encoded)[:, 1]
# auc_rf = roc_auc_score(y_test, rf_pred_proba)
# auc_scores['Random Forest'] = roc_auc_score(y_test, rf_pred_proba)

# 2. LightGBM
start_time = time.time()
lgb_train = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_columns)
lgb_test = lgb.Dataset(X_test, label=y_test, reference=lgb_train)

lgb_params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 50,
    'max_depth': 10,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1
}

lgb_model = lgb.train(lgb_params, lgb_train, num_boost_round=1000, valid_sets=[lgb_test])
training_times['LightGBM'] = time.time() - start_time
lgb_pred_proba = lgb_model.predict(X_test)
auc_scores['LightGBM'] = roc_auc_score(y_test, lgb_pred_proba)


# 3. XGBoost
start_time = time.time()
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='auc', random_state=42)
xgb_model.fit(X_train, y_train)
training_times['XGBoost'] = time.time() - start_time
xgb_pred_proba = xgb_model.predict_proba(X_test)[:, 1]
auc_scores['XGBoost'] = roc_auc_score(y_test, xgb_pred_proba)

# 4. 앙상블
ensemble_pred_proba = (lgb_pred_proba + xgb_pred_proba) / 2
auc_scores['Ensemble'] = roc_auc_score(y_test, ensemble_pred_proba)
training_times['Ensemble'] = 0

# 결과 출력
print("________________________________________________________________________________")
print("\n모델별 ROC-AUC 점수 및 학습 시간:")
for model in auc_scores:
    print(f"{model}: ROC-AUC = {auc_scores[model]:.4f}, 학습 시간 = {training_times[model]:.2f}초")
print("________________________________________________________________________________ \n")

# ________________REAL TRAIN_____________

# etc_pred_proba = ETC_model.predict_proba(X_test_encoded)[:, 1]
# rf_pred_proba = rf_model.predict_proba(X_test_encoded)[:, 1]

lgb_train = lgb.Dataset(X_train_encoded, label=y, categorical_feature=categorical_columns)
lgb_model = lgb.train(lgb_params, lgb_train, num_boost_round=1000)
lgb_pred_proba = lgb_model.predict(X_test_encoded)

xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='auc', random_state=42)
xgb_model.fit(X_train_encoded, y)
xgb_pred_proba = xgb_model.predict_proba(X_test_encoded)[:, 1]


# 결과 제출 file
sample_submission = pd.read_csv('./data/sample_submission.csv')

# 앙상블 (LightGBM과 XGBoost 예측값 평균)
ensemble_pred_proba = (lgb_pred_proba + xgb_pred_proba) / 2
sample_submission['probability'] = ensemble_pred_proba
sample_submission.to_csv('./submit/ensemble_submit.csv', index=False)

# sample_submission['probability'] = etc_pred_proba
# sample_submission.to_csv('./etc_submit.csv', index=False)
# sample_submission['probability'] = rf_pred_proba   
# sample_submission.to_csv('./rf_submit.csv', index=False)
sample_submission['probability'] = lgb_pred_proba
sample_submission.to_csv('./submit/lgb_submit.csv', index=False)

sample_submission['probability'] = xgb_pred_proba
sample_submission.to_csv('./submit/xgb_submit.csv', index=False)

print(sample_submission.head(5))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 171209, number of negative: 171012
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020332 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3929
[LightGBM] [Info] Number of data points in the train set: 342221, number of used features: 62
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500288 -> initscore=0.001151
[LightGBM] [Info] Start training from score 0.001151


/home/najo/.conda/envs/dip/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [20:36:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


________________________________________________________________________________

모델별 ROC-AUC 점수 및 학습 시간:
LightGBM: ROC-AUC = 0.8955, 학습 시간 = 13.03초
XGBoost: ROC-AUC = 0.9070, 학습 시간 = 0.93초
Ensemble: ROC-AUC = 0.9074, 학습 시간 = 0.00초
Split TEST ROC-AUC 점수 

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 190123, number of negative: 190123
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020479 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3920
[LightGBM] [Info] Number of data points in the train set: 380246, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


/home/najo/.conda/envs/dip/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [20:36:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


           ID  probability
0  TEST_00000     0.000806
1  TEST_00001     0.002128
2  TEST_00002     0.136237
3  TEST_00003     0.151065
4  TEST_00004     0.515323


## 피쳐 제거 이후 학습...

In [ ]:
# 피처 중요도 추출
feature_importances = xgb_model.feature_importances_
feature_names = X.columns
feature_importances = pd.Series(xgb_model.feature_importances_, index=X.columns).sort_values(ascending=False)

# 중요도 기준으로 상위 10개만 선택
top_features = feature_importances[feature_importances < 0.002].index  # 중요도가 0.01 이상인 컬럼만 선택
X_selected = X[top_features]

# 선택된 컬럼으로 학습 진행
print(f"선택된 컬럼: {list(top_features)}")
print(f"선택된 컬럼 개수: {len(list(top_features))}")

# 선택된 컬럼
selected_columns = ['여성 주 불임 원인', '해동된 배아 수', '총 시술 횟수', '부부 주 불임 원인', 
                   '미세주입에서 생성된 배아 수', '해동 난자 수', '기증자 정자와 혼합된 난자 수', 
                   '미세주입 후 저장된 배아 수', '불명확 불임 원인', '대리모 여부', 
                   '남성 주 불임 원인', '난자 혼합 경과일', '저장된 신선 난자 수', 
                   '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '불임 원인 - 자궁경부 문제', 
                   '난자 해동 경과일', '난자 채취 경과일', '불임 원인 - 여성 요인', 
                   '불임 원인 - 정자 면역학적 요인']
selected_columns = ['저장된 신선 난자 수', '불임 원인 - 정자 운동성', '난자 채취 경과일', '난자 해동 경과일',
                     '불임 원인 - 자궁경부 문제', '불임 원인 - 여성 요인', '불임 원인 - 정자 면역학적 요인',
                       '동결 배아 사용 여부']


# 선택된 컬럼 제거
X_train_new = X_train.drop(columns=selected_columns, errors='ignore')
X_test_new = X_test.drop(columns=selected_columns, errors='ignore')  # 테스트 데이터도 동일하게 처리

# 모델 학습 및 평가
auc_scores_new = {}

# XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='auc', random_state=42)
xgb_model.fit(X_train_new, y_train)
xgb_pred_proba = xgb_model.predict_proba(X_test_new)[:, 1]
auc_scores_new['XGBoost'] = roc_auc_score(y_test, xgb_pred_proba)

# 결과 출력
print("제외된 컬럼 이후 모델별 ROC-AUC 점수:")
for model, auc in auc_scores_new.items():
    print(f"{model}: {auc:.4f}")
